In [1]:
!nvidia-smi

Mon May 10 16:59:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
print(torch.__version__)

1.8.1+cu101


In [3]:
!cp drive/MyDrive/zindi_nlp/MalawiNews/*.csv .

In [4]:
!pip install git+https://github.com/huggingface/transformers.git

#!pip install transformers==4.5.0

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-t7pl8olf
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-t7pl8olf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/a1/88/7b1e45720ecf59c6c6737ff332f41c955963090a18e72acbcbeac6b25e86/huggingface_hub-0.0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/75/ee/67241dc87f266093c533a2d4d3d69438e57d7a90abb216fa076e7d475d4a/sacremoses-0.0.45-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ae/04/5b870f26a858552025a62f1649c20d29d2672c02ff3c3fb4c688ca46467a/tokenizers-0.10.2-cp37-cp37m-manylinux2010_x86_64.whl
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2217008 sha256=5bb8c2d056475e1112be6b27827ebae24fa893f0497ee46ac65671062b392929
  Stored in di

In [5]:
!pip install datasets sentencepiece 

     |████████████████████████████████| 225kB 16.9MB/s 
     |████████████████████████████████| 1.2MB 28.3MB/s 
     |████████████████████████████████| 245kB 64.9MB/s 
     |████████████████████████████████| 112kB 69.2MB/s 


In [6]:
!pip install emoji

     |████████████████████████████████| 133kB 12.1MB/s 


In [7]:
! rm -r transformers

rm: cannot remove 'transformers': No such file or directory


In [8]:
! git clone https://github.com/huggingface/transformers.git
#  && cd transformers && git checkout 4906a29 
# && cd transformers && git checkout bae0c79

Cloning into 'transformers'...
remote: Enumerating objects: 72233, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 72233 (delta 118), reused 153 (delta 53), pack-reused 71959
Receiving objects: 100% (72233/72233), 55.04 MiB | 23.42 MiB/s, done.
Resolving deltas: 100% (51253/51253), done.


In [9]:
import pandas as pd
import numpy as np
import warnings
from tqdm.auto import tqdm
warnings.simplefilter("ignore")

data = pd.read_csv("Train.csv").Text.tolist() + pd.read_csv("Test.csv").Text.tolist()
print(len(data))
print(len(pd.read_csv("Train.csv")), len(pd.read_csv("Test.csv")))

2056
1436 620


In [10]:
texts = []
for x in tqdm(data):
  texts.extend(x.strip().split("\n"))


In [11]:
#%load_ext tensorboard

In [12]:
with open("datas.txt", "w") as f:
  for line in texts:
    f.write(line.strip() + "\n")


In [13]:
#%tensorboard --logdir runs

In [ ]:
!python transformers/examples/pytorch/language-modeling/run_mlm.py --model_name_or_path "xlm-roberta-large" --fp16 --max_seq_length 200 --gradient_accumulation 4  \
  --train_file "datas.txt" --validation_file "datas.txt"  --do_train --per_device_train_batch_size 4 --save_steps 10000 --do_eval --line_by_line --seed 42 \
  --learning_rate 1e-5 --mlm_probability 0.2 --num_train_epochs 1000  --output_dir drive/MyDrive/zindi_nlp/MalawiNews/pretrain/xlm-r-large/ --preprocessing_num_workers=4\
  # --pad_to_max_length True   #  #"google/electra-small-generator" \   transformers/examples/xla_spawn.py --num_cores 8  

2021-05-10 16:59:37.548507: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/10/2021 16:59:39 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
05/10/2021 16:59:39 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=drive/MyDrive/zindi_nlp/MalawiNews/pretrain/xlm-r-large/, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=8, gradient_accumulation_steps=4, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1000.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/May10_16-59-39_2d45db7298d1, logging_strategy=I

In [ ]:
!python transformers/examples/language-modeling/run_mlm.py -h